In [ ]:
#unlike other API notebooks, no need to import a .py file with API token info
#for kaggle, lives locally in ~/.kaggle/kaggle.json file

To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (https://www.kaggle.com/<username>/account) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials. Place this file in the location ~/.kaggle/kaggle.json (on Windows in the location C:\Users\<Windows-username>\.kaggle\kaggle.json - you can check the exact location, sans drive, with echo %HOMEPATH%). You can define a shell environment variable KAGGLE_CONFIG_DIR to change this location to $KAGGLE_CONFIG_DIR/kaggle.json (on Windows it will be %KAGGLE_CONFIG_DIR%\kaggle.json).

In [1]:
import kaggle

In [8]:
#there are Pythonic options, but seems easiest to just run terminal commands from within notebook

#example:
! kaggle competitions list -s health

ref                                            deadline             category       reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ----------  ---------  ---------  --------------  
hhp                                            2013-04-04 07:00:00  Featured     $500,000       1350           False  
datasciencebowl                                2015-03-16 23:59:00  Featured     $175,000       1049           False  
stanford-covid-vaccine                         2020-10-06 23:59:00  Research      $25,000       1636           False  
hubmap-kidney-segmentation                     2021-05-10 23:59:00  Research      $60,000       1200           False  
osic-pulmonary-fibrosis-progression            2020-10-06 23:59:00  Featured      $55,000       2097           False  
histopathologic-cancer-detection               2019-03-30 23:59:00  Playground  Knowledge       1149           False  
rsna-str-pulmonary-embolism-detection          2

In [10]:
#search datasets for term
! kaggle datasets list -s "machine learning"

#this returns by default one page of results at a time (matches manual search of datasets page 1)
#need to extract "ref" variable, which is <owner>/<dataset-name>

ref                                                   title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
kaggle/kaggle-survey-2018                             2018 Kaggle Machine Learning & Data Science Survey    4MB  2018-11-03 22:35:07          15135        972  0.85294116       
kaggle/kaggle-survey-2017                             2017 Kaggle Machine Learning & Data Science Survey    4MB  2017-10-27 22:03:03          22642        825  0.8235294        
alopez247/pokemon                                     Pokémon for Data Mining and Machine Learning        715KB  2017-03-05 15:01:26          10332        237  0.85294116       
kashnitsky/mlcourse                                   mlcourse.ai                                          51M

In [ ]:
#searching datasets is a start, but we'll also want to search notebooks, competitions, etc
#looks like can search datasets and kernels
#and get related files for each

In [11]:
#once get list of files in format <owner>/<dataset-name>, can get list of files
#example:
! kaggle datasets files kaggle/kaggle-survey-2018 

name                         size  creationDate         
---------------------------  ----  -------------------  
SurveySchema.csv              8KB  2018-11-03 22:35:07  
multipleChoiceResponses.csv  39MB  2018-11-03 22:35:07  
freeFormResponses.csv         1MB  2018-11-03 22:35:07  


In [27]:
#can get metadata for a dataset
#default downloads into current working directory
#need a way to store in memory and iterate for all files
! kaggle datasets metadata -p /Kaggle_results/ kaggle/kaggle-survey-2018 

#this currently barfs, not sure why - something with where putting output,b ut not sure why
#TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

Traceback (most recent call last):
  File "c:\users\slabou\appdata\local\continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\slabou\appdata\local\continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\slabou\AppData\Local\Continuum\anaconda3\Scripts\kaggle.exe\__main__.py", line 7, in <module>
  File "c:\users\slabou\appdata\local\continuum\anaconda3\lib\site-packages\kaggle\cli.py", line 67, in main
    out = args.func(**command_args)
  File "c:\users\slabou\appdata\local\continuum\anaconda3\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 1065, in dataset_metadata_cli
    meta_file = self.dataset_metadata(dataset, path)
  File "c:\users\slabou\appdata\local\continuum\anaconda3\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 1050, in dataset_metadata
    metadata = Metadata(result['info'])
  File "c:\users\slabou\appdata\local\continuum\anaconda3\lib\site-pack